In [1]:
import os
import pandas as pd
import numpy as np
import json
import pickle
from collections import defaultdict
from pathlib import Path
from sklearn.metrics import ndcg_score, dcg_score
import matplotlib.pyplot as plt
import seaborn as sns

import torch

import os, sys
parentPath = os.path.abspath("..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)
    
from src.data import load_source, slice_d
from src.config import Config, get_option_fallback
from src.path import get_best_model_paths, get_exp_paths, get_report_path, load_json, load_rep_cfg
from knowledge_tracing.trainer import Trainer

/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
projectdir = Path('/code')
assert projectdir.exists()

In [21]:
results = [
    'output/20_0218_edm2020_asmt0910/pre_dummy_epoch_size0.auto/report/20200220-0823/ksdkt.json',
    'output/20_0218_edm2020_asmt0910/pre_dummy_epoch_size10.auto/report/20200220-0535/ksdkt.json',
    'output/20_0218_edm2020_asmt0910/pre_dummy_epoch_size150.auto/report/20200220-0431/ksdkt.json',
    
    'output/20_0222_edm2020_asmt15/pre_dummy_epoch_size0.auto/report/20200223-1405/report.json',
    'output/20_0222_edm2020_asmt15/pre_dummy_epoch_size10.auto/report/20200223-0724/report.json',
    'output/20_0222_edm2020_asmt15/pre_dummy_epoch_size150.auto/report/20200223-0354/report.json',
]

In [27]:
df = pd.DataFrame(columns=['dataset', 'model', 'pre', 'bestepoch', 'AUC', 'std'])
df = pd.DataFrame(columns=['dataset', 'pre', 'AUC', 'std'])

for res_path in results:
    with open(projectdir / res_path, 'r') as f:
        res = json.load(f)
    row = {
        'dataset':   res['config']['source_data'],
#         'model':     res['config']['model_name']+'/'+res['config']['exp_name'],
        'pre':       res['config']['pre_dummy_epoch_size'],
#         'bestepoch': np.mean(list(res['best']['auc_epoch'].values())),
        'AUC':       np.mean(list(res['best']['auc'].values())),
        'std':       np.std(list(res['best']['auc'].values())),
    }
    df = df.append(row, ignore_index=True)
    
df

,dataset,pre,AUC,std
0,original_ASSISTmentsSkillBuilder0910,0,0.827841,0.032821
1,original_ASSISTmentsSkillBuilder0910,10,0.831144,0.031850
2,original_ASSISTmentsSkillBuilder0910,150,0.830445,0.031801
3,2015_100_skill_builders_main_problems,0,0.690579,0.003041
4,2015_100_skill_builders_main_problems,10,0.691871,0.002877
5,2015_100_skill_builders_main_problems,150,0.692713,0.003017


In [28]:
print(df.to_latex())

\begin{tabular}{lllrr}
\toprule
{} &                                dataset &  pre &       AUC &       std \\
\midrule
0 &   original\_ASSISTmentsSkillBuilder0910 &    0 &  0.827841 &  0.032821 \\
1 &   original\_ASSISTmentsSkillBuilder0910 &   10 &  0.831144 &  0.031850 \\
2 &   original\_ASSISTmentsSkillBuilder0910 &  150 &  0.830445 &  0.031801 \\
3 &  2015\_100\_skill\_builders\_main\_problems &    0 &  0.690579 &  0.003041 \\
4 &  2015\_100\_skill\_builders\_main\_problems &   10 &  0.691871 &  0.002877 \\
5 &  2015\_100\_skill\_builders\_main\_problems &  150 &  0.692713 &  0.003017 \\
\bottomrule
\end{tabular}

